# TensorFlow Lite Flatbuffer Manipulation Example

It's possible to read, modify, and write TensorFlow Lite model files from Python. This notebook shows you how.
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tinyMLx/colabs/blob/master/4-4-8-Flatbuffers.ipynb
"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tinyMLx/colabs/blob/master/4-4-8-Flatbuffers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Software installation

The goal is to build a set of Python classes that represent the types stored inside the TFL Flatbuffer files. To do this we need several dependencies:
 - The 'flatc' compiler, that converts the model format stored in a text schema to Python accessor classes.
 - The text schema itself, describing the model format.
 - The Flatbuffer Python library that the accessor classes rely on.

The 'flatc' compiler isn't available as a binary download, so we need to build it from source. The version has to match the Flatbuffer Python library on the system, or the resulting generated code won't work, since it will be trying to use a different API. The FB Python library is at version 1.12.0, so we make sure we download the tagged snapshot of the source at that same version from GitHub.

### Install Flatbuffer Python Library

We should already have version 1.12.0 by default on this Colab, but use pip to ensure it's installed, and then import it so it's available from Python.

In [1]:
!pip3 install flatbuffers==1.12.0
import flatbuffers

### Build the 'flatc' Compiler

We need 'flatc' to generate the Python accessor classes we'll use to read and write the serialized files, and since it's not easily available as a binary, we grab the source code from the right version and build it directly. **This will take a few minutes.**

Once the 'flatc' binary has been built, copy it to the `/usr/local/bin` folder so that it's easily accessible as a command.

In [2]:
# Build and install the Flatbuffer compiler.
%cd /content/
!rm -rf flatbuffers*
!curl -L "https://github.com/google/flatbuffers/archive/v1.12.0.zip" -o flatbuffers.zip
!unzip -q flatbuffers.zip
!mv flatbuffers-1.12.0 flatbuffers
%cd flatbuffers
!cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
!make -j 8
!cp flatc /usr/local/bin/

[Errno 2] No such file or directory: '/content/'
/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   124  100   124    0     0    116      0  0:00:01  0:00:01 --:--:--   116
100 1463k    0 1463k    0     0   419k      0 --:--:--  0:00:03 --:--:--  783k
/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro/flatbuffers
CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler

[ 74%] Building CXX object CMakeFiles/flatsamplebfbs.dir/samples/sample_bfbs.cpp.o
[ 75%] Building CXX object CMakeFiles/flattests.dir/src/idl_gen_text.cpp.o
[ 76%] Building CXX object CMakeFiles/flattests.dir/src/reflection.cpp.o
[ 77%] Building CXX object CMakeFiles/flattests.dir/src/util.cpp.o
[ 78%] Building CXX object CMakeFiles/flattests.dir/src/idl_gen_fbs.cpp.o
[ 80%] Building CXX object CMakeFiles/flattests.dir/tests/test.cpp.o
[ 81%] Building CXX object CMakeFiles/flattests.dir/tests/test_assert.cpp.o
[ 82%] Building CXX object CMakeFiles/flattests.dir/tests/test_builder.cpp.o
[ 83%] Building CXX object CMakeFiles/flattests.dir/tests/native_type_test_impl.cpp.o
[ 84%] Building CXX object CMakeFiles/flattests.dir/src/code_generators.cpp.o
[ 85%] Linking CXX executable flatsampletext
[ 87%] Built target flatsampletext
[ 88%] Linking CXX executable flatsamplebfbs
[ 89%] Built target flatsamplebfbs
[ 90%] Linking CXX executable flattests
[100%] Built target flattests
cp: cannot c

### Fetch the Schema

The schema is [a text file that describes the layout of the model file format](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/schema/schema_v3.fbs), and it's part of the TensorFlow source code, so we need to fetch the latest version from GitHub.

In [4]:
%cd /content/
!rm -rf tensorflow
!git clone --depth 1 https://github.com/tensorflow/tensorflow

[Errno 2] No such file or directory: '/content/'
/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro/flatbuffers
Cloning into 'tensorflow'...
remote: Enumerating objects: 24025, done.
remote: Counting objects: 100% (24025/24025), done.
remote: Compressing objects: 100% (17902/17902), done.
remote: Total 24025 (delta 8850), reused 10365 (delta 5555), pack-reused 0
Receiving objects: 100% (24025/24025), 57.45 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (8850/8850), done.
Updating files: 100% (24541/24541), done.


### Generate the Python Accessor Classes

The 'flatc' compiler takes the information from the text schema, and creates Python code to read and write the data held inside the serialized Flatbuffer file. The Python classes are written into the `tflite` folder, with names like `Model.py`, and define classes like `ModelT` that contain members that you can use to read or write the contents of the data structures defined by the schema.

In [5]:
!flatc --python --gen-object-api tensorflow/tensorflow/lite/schema/schema_v3.fbs

### Model Loading and Saving

These are some simple wrapper functions that demonstrate how to load data from a file, turn it into a `ModelT` Python object that can be modified, and then save it out to a new file.

We have to do a little bit of hackery with the Python search paths so that the class files we generated from the schema can be imported.

In [6]:
import sys
# This hackery allows us to import the Python files we've just generated.
sys.path.append("/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro/flatbuffers/tflite")
import Model

def load_model_from_file(model_filename):
  with open(model_filename, "rb") as file:
    buffer_data = file.read()
  model_obj = Model.Model.GetRootAsModel(buffer_data, 0)
  model = Model.ModelT.InitFromObj(model_obj)
  return model

def save_model_to_file(model, model_filename):
  builder = flatbuffers.Builder(1024)
  model_offset = model.Pack(builder)
  builder.Finish(model_offset, file_identifier=b'TFL3')
  model_data = builder.Output()
  with open(model_filename, 'wb') as out_file:
    out_file.write(model_data)

## Download an Example Model

This pulls down a trained model from the speech commands tutorial that we can use to test the Flatbuffer loading code.

In [7]:
!curl -O 'https://storage.googleapis.com/download.tensorflow.org/models/tflite/micro/speech_commands_model_2020_04_27.zip'
!unzip speech_commands_model_2020_04_27.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78855  100 78855    0     0  70155      0  0:00:01  0:00:01 --:--:-- 70218
Archive:  speech_commands_model_2020_04_27.zip
  inflating: speech_commands_model/speech_commands_model_float.tflite  
  inflating: speech_commands_model/speech_commands_model_int8.tflite  


## Load, Modify, and Save a Model

The code below loads the float model, applies a small change to the float weights, and saves it out again. In this case we're just changing the contents of the weights, but any of the other properties of the model can be added, removed, or modified, including tensors, ops, and metadata.

I've found the easiest way to understand the syntax used and data available is to look at the generated classes in `tflite/` and [the text schema that describes the format](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/schema/schema_v3.fbs).

In [9]:
# Use numpy to make the manipulation of the weight arrays easier.
import numpy as np

# Load the float model we downloaded as a ModelT object.
model = load_model_from_file('/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro/flatbuffers/speech_commands_model/speech_commands_model_float.tflite')

# Loop through all the weights held in the model.
for buffer in model.buffers:
  # Skip missing or small weight arrays.
  if buffer.data is not None and len(buffer.data) > 1024:
    # Pull the weight array from the model, and cast it to 32-bit floats since
    # we know that's the type for all the weights in this model. In a real
    # application we'd need to check the data type from the tensor information
    # stored in the model.subgraphs
    original_weights = np.frombuffer(buffer.data, dtype=np.float32)

    # This is the line where the weights are altered.
    # Try replacing it with your own version, for example:
    # munged_weights = np.add(original_weights, 0.002)
    munged_weights = np.round(original_weights * (1/0.02)) * 0.02

    # Write the altered data back into the model.    
    buffer.data = munged_weights.tobytes()

# Save the ModelT object as a TFL Flatbuffer file.
save_model_to_file(model, '/home/ankdesh/explore/LearnTry-ML/Tensorflow/tflite/course-edx-tinyml3/1.4-TensorFlow Lite Micro/flatbuffers/speech_commands_model/speech_commands_model_modified.tflite')

## Evaluating the impact of your changes

### Setup KWS infrastructure

To evaluate the impact of your changes on the accuracy of the speech model we need to load a test data set and some utility classes to read the files and convert them into the right input form for the network.

**The full dataset is several gigabytes in size, so it may take a few minutes to download.**

In [18]:
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All the other words will be used to train an "unknown" label and silent
# audio data with no spoken words will be used to train a "silence" label.
WANTED_WORDS = "yes,no"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=12000,3000 and LEARNING_RATE=0.001,0.0001
# will run 12,000 training loops in total, with a rate of 0.001 for the first
# 8,000, and 0.0001 for the final 3,000.
TRAINING_STEPS = "12000,3000"
LEARNING_RATE = "0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE =20
MODEL_ARCHITECTURE = 'tiny_conv' # Other options include: single_fc, conv,
                      # low_latency_conv, low_latency_svdf, tiny_embedding_conv

# Constants used during training only
VERBOSITY = 'WARN'
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
DATASET_DIR =  'dataset/'
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.

SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

DATA_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE, CLIP_DURATION_MS, WINDOW_SIZE_MS,
    WINDOW_STRIDE, FEATURE_BIN_COUNT, PREPROCESS)
audio_processor = input_data.AudioProcessor(
    DATA_URL, DATASET_DIR,
    SILENT_PERCENTAGE, UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE, model_settings, LOGS_DIR)

>> Downloading speech_commands_v0.02.tar.gz 100.0%
INFO:tensorflow:Successfully downloaded speech_commands_v0.02.tar.gz (2428923189 bytes)


In [19]:
import tensorflow as tf
# define our helper function to test the model accuracy
def test_model_accuracy(model_filename):
  with tf.compat.v1.Session() as sess:
    test_data, test_labels = audio_processor.get_data(
        -1, 0, model_settings, 0, 0,
        0, 'testing', sess)

  interpreter = tf.lite.Interpreter(model_filename)
  interpreter.allocate_tensors()

  input_index = interpreter.get_input_details()[0]["index"]

  output_index = interpreter.get_output_details()[0]["index"]
  model_output = interpreter.tensor(output_index)

  correct_predictions = 0
  for i in range(len(test_data)):
    current_input = test_data[i]
    current_label = test_labels[i]
    flattened_input = np.array(current_input.flatten(), dtype=np.float32).reshape(1, 1960)
    interpreter.set_tensor(input_index, flattened_input)
    interpreter.invoke()
    top_prediction = model_output()[0].argmax()
    if top_prediction == current_label:
      correct_predictions += 1

  print('Accuracy is %f%% (N=%d)' % ((correct_predictions * 100) / len(test_data), len(test_data)))

### Test your Models

Finally we can test both the standard and your modified models!

**Float model evaluation**

You should see ~91% accuracy with a 67KB model. Note: the exact accuracy may vary by a percent or two as it tests the model on a random sampling of the dataset and therefore sometimes gets particularly lucky or unlucky!

In [ ]:
!ls -lah /content/speech_commands_model/speech_commands_model_float.tflite
test_model_accuracy('/content/speech_commands_model/speech_commands_model_float.tflite')

**Modified model evaluation**

Test the impact of your changes!

In [ ]:
!ls -lah /content/speech_commands_model/speech_commands_model_modified.tflite
test_model_accuracy('/content/speech_commands_model/speech_commands_model_modified.tflite')

If you'd like to try more modified models just scroll back up to the **Load, Modify, and Save a Model** section and re-run the modified model generation step and then skip back down to the above to re-test!